# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
from scipy import stats
import seaborn as sns

In [4]:
POSTGRES_ADDRESS = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' 
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = 'lhl_student' 
POSTGRES_PASSWORD = 'lhl_student' 
POSTGRES_DBNAME='mid_term_project'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, 
                                                                                        password=POSTGRES_PASSWORD, 
                                                                                        ipaddress=POSTGRES_ADDRESS, 
                                                                                        port=POSTGRES_PORT, 
                                                                                        dbname=POSTGRES_DBNAME)) 

cnx = create_engine(postgres_str)

In [5]:
df_flight=pd.read_sql_query('''SELECT * FROM flights ORDER BY RANDOM() LIMIT 10000;''', cnx)

In [142]:
df_flight.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-11-28,UA,UA,UA,1436,UA,N37290,1436,13930,ORD,...,220.0,230.0,197.0,1.0,1250.0,23.172619,4.725108,16.795996,0.048701,25.610931
1,2018-01-03,HA,HA,HA,181,HA,N485HA,181,12402,ITO,...,55.0,50.0,36.0,1.0,216.0,23.172619,4.725108,16.795996,0.048701,25.610931
2,2018-02-15,AA,AA_CODESHARE,AA,3477,MQ,N542EA,3477,11066,CMH,...,96.0,92.0,64.0,1.0,296.0,69.000000,0.000000,0.000000,0.000000,0.000000
3,2018-07-27,B6,B6,B6,730,B6,N527JB,730,10732,BQN,...,173.0,172.0,151.0,1.0,1129.0,23.172619,4.725108,16.795996,0.048701,25.610931
4,2018-03-27,AS,AS,AS,815,AS,N585AS,815,14747,SEA,...,398.0,359.0,343.0,1.0,2701.0,23.172619,4.725108,16.795996,0.048701,25.610931


## Data Cleaning


In [143]:
df_flight.isnull().sum()

fl_date                0
mkt_unique_carrier     0
branded_code_share     0
mkt_carrier            0
mkt_carrier_fl_num     0
op_unique_carrier      0
tail_num               0
op_carrier_fl_num      0
origin_airport_id      0
origin                 0
origin_city_name       0
dest_airport_id        0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
cancelled              0
diverted               0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
flights                0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [144]:
df_flight.isnull().sum()

fl_date                0
mkt_unique_carrier     0
branded_code_share     0
mkt_carrier            0
mkt_carrier_fl_num     0
op_unique_carrier      0
tail_num               0
op_carrier_fl_num      0
origin_airport_id      0
origin                 0
origin_city_name       0
dest_airport_id        0
dest                   0
dest_city_name         0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
cancelled              0
diverted               0
dup                    0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
flights                0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [145]:
# replacing the missing values with their mean
carr_delay_mean=df_flight['carrier_delay'].astype('float').mean(axis=0)
wea_delay_mean=df_flight['weather_delay'].astype('float').mean(axis=0)
nas_delay_mean=df_flight['nas_delay'].astype('float').mean(axis=0)
sec_delay_mean=df_flight['security_delay'].astype('float').mean(axis=0)
aircraft_delay_mean=df_flight['late_aircraft_delay'].astype('float').mean(axis=0)


df_flight['carrier_delay'].replace(np.nan, carr_delay_mean, inplace=True)
df_flight['weather_delay'].replace(np.nan, wea_delay_mean, inplace=True)
df_flight['nas_delay'].replace(np.nan, nas_delay_mean, inplace=True)
df_flight['security_delay'].replace(np.nan, sec_delay_mean, inplace=True)
df_flight['late_aircraft_delay'].replace(np.nan, aircraft_delay_mean, inplace=True)

In [146]:
df_flight.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled', 'diverted', 'dup',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [134]:
df=pd.read_csv('flights_rand-Copy1.csv')
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [135]:

df_flights = df.drop(['dep_time', 'actual_elapsed_time', 'air_time','dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'arr_time', 'dest', 'dest_city_name','origin', 'origin_city_name', 'branded_code_share', 'carrier_delay','weather_delay','nas_delay','security_delay', 'late_aircraft_delay', 'first_dep_time', 'total_add_gtime','longest_add_gtime','cancelled', 'cancellation_code', 'diverted', 'dup', 'flights', 'no_name'], axis = 1)

In [136]:
df_flights.columns


Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 'mkt_carrier_fl_num',
       'op_unique_carrier', 'tail_num', 'op_carrier_fl_num',
       'origin_airport_id', 'dest_airport_id', 'crs_dep_time', 'crs_arr_time',
       'arr_delay', 'crs_elapsed_time', 'distance'],
      dtype='object')

In [93]:
dep_time_mode = df_flights['dep_time'].mode()
arr_time_mode = df_flights['arr_time'].mode()

tail_number_mode = df_flights['tail_num'].mode()

In [94]:

df_flights['arr_time'] = df_flights['arr_time'].fillna(int(arr_time_mode))
df_flights['dep_time'] = df_flights['dep_time'].fillna(int(dep_time_mode))

df_flights['tail_num'] = df_flights['tail_num'].fillna(str(tail_number_mode[0]))

In [95]:
df_flights['dep_delay'] = df_flights['dep_delay'].fillna(df_flights['dep_delay'].mean())
df_flights['taxi_in'] = df_flights['taxi_in'].fillna(df_flights['taxi_in'].mean())
df_flights['arr_delay'] = df_flights['arr_delay'].fillna(df_flights['arr_delay'].mean())
df_flights['actual_elapsed_time'] = df_flights['actual_elapsed_time'].fillna(df_flights['actual_elapsed_time'].mean())
df_flights['air_time'] = df_flights['air_time'].fillna(df_flights['air_time'].mean())
df_flights['taxi_out'] = df_flights['taxi_out'].fillna(df_flights['taxi_out'].mean())

In [96]:
df_flights=df_flights.dropna()
df_flights.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
0,2018-07-27,DL,5799,CP,N615CZ,5799,14869,12280,1100,1056.0,-4.0,18.0,5.0,1204,1202.0,-2.0,64.0,66.0,43.0,188.0
1,2018-10-08,UA,6129,YV,N88325,6129,12266,11540,955,1016.0,21.0,22.0,2.0,1051,1111.0,20.0,116.0,115.0,91.0,667.0
2,2018-10-31,WN,809,WN,N7841A,809,13232,11433,1640,1638.0,-2.0,11.0,6.0,1850,1831.0,-19.0,70.0,53.0,36.0,228.0
3,2019-10-15,AA,1830,AA,N556UW,1830,12889,11057,750,750.0,0.0,20.0,15.0,1508,1506.0,-2.0,258.0,256.0,221.0,1916.0
4,2018-06-19,AS,3300,OO,N176SY,3300,14831,10423,938,932.0,-6.0,14.0,5.0,1514,1458.0,-16.0,216.0,206.0,187.0,1476.0


In [97]:
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'], errors='coerce')
df_flights['month'] = df_flights['fl_date'].dt.month
df_flights['day_of_week'] = df_flights['fl_date'].dt.dayofweek
df_flights['day_of_month'] = df_flights['fl_date'].dt.day
df_flights['year'] = df_flights['fl_date'].dt.year


In [98]:
df_flights=df_flights.drop('fl_date', axis=1)

In [99]:
#make the types categories
df_flights["mkt_carrier"] = df_flights["mkt_carrier"].astype("category")
df_flights["op_unique_carrier"] = df_flights["op_unique_carrier"].astype("category")
df_flights["tail_num"] = df_flights["tail_num"].astype("category")
df_flights["op_carrier_fl_num"] = df_flights["op_carrier_fl_num"].astype("category")
df_flights["origin_airport_id"] = df_flights["origin_airport_id"].astype("category")
df_flights["dest_airport_id"] = df_flights["dest_airport_id"].astype("category")
df_flights["mkt_carrier_fl_num"] = df_flights["mkt_carrier_fl_num"].astype("category")

In [100]:
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

df_flights['mkt_carrier'] = encoder.fit_transform(df_flights[['mkt_carrier']])
df_flights['mkt_carrier_fl_num'] = encoder.fit_transform(df_flights[['mkt_carrier_fl_num']])
df_flights['op_unique_carrier'] = encoder.fit_transform(df_flights[['op_unique_carrier']])
df_flights['tail_num'] = encoder.fit_transform(df_flights[['tail_num']])
df_flights['op_carrier_fl_num'] = encoder.fit_transform(df_flights[['op_carrier_fl_num']])
df_flights['origin_airport_id'] = encoder.fit_transform(df_flights[['origin_airport_id']])
df_flights['dest_airport_id'] = encoder.fit_transform(df_flights[['dest_airport_id']])

In [123]:
#Converting to air_time and arr_delay to category
def flight_duration(x):
    if x <=180:
        return '1'
    elif x >180 and x<360:
        return '2'
    elif x>360:
        return '3'

df_flights['flight_duration_type']=df_flights['air_time'].apply(lambda x: flight_duration(x))

df_flights['flight_duration_type'].value_counts()

1    8666
2    1295
3      38
Name: flight_duration_type, dtype: int64

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [37]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [38]:
df_flights.columns

Index(['mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id',
       'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off',
       'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time', 'arr_delay',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance',
       'month', 'day_of_week', 'day_of_month', 'year', 'flight_duration_type',
       'arr_delay_cat'],
      dtype='object')

In [39]:
features=df_flights[['mkt_carrier_fl_num','origin_airport_id', 'dest_airport_id','crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out', 'taxi_in', 'crs_arr_time', 'arr_time', 'arr_delay',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance',
       'month', 'day_of_week', 'day_of_month', 'year', 'flight_duration_type']]

pca=PCA(n_components=3)
components=pca.fit_transform(features)

ValueError: could not convert string to float: 'Short'

In [86]:
import plotly.express as px

fig=px.scatter(components, x=0, y=1, color=df_flights['month'])
fig.show()

NameError: name 'components' is not defined

In [102]:
df_flights.dtypes

mkt_carrier            float64
mkt_carrier_fl_num     float64
op_unique_carrier      float64
tail_num               float64
op_carrier_fl_num      float64
origin_airport_id      float64
dest_airport_id        float64
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
distance               float64
month                    int64
day_of_week              int64
day_of_month             int64
year                     int64
arr_delay_cat            int64
dtype: object

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [103]:
import pickle

In [124]:
#Spliting the data into train and test

X=np.array(df_flights.drop('dep_delay', axis=1))
y=np.array(df_flights['dep_delay'])

In [125]:
y

array([-4., 21., -2., ...,  9., 44., -7.])

In [126]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = (y - y.mean()) / y.std()

In [127]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=100)

In [128]:
from sklearn.preprocessing import LabelEncoder
lab_enc=LabelEncoder()
y_train=lab_enc.fit_transform(y_train)

In [129]:
#Logistics Regression
from sklearn.linear_model import LogisticRegression

model_log=LogisticRegression()
model_log.fit(X_train, y_train)
y_pred_log=model_log.predict(X_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [119]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred_log)


-2421.3721103160897

In [120]:
# Linear Regression

from sklearn.linear_model import LinearRegression

model_linear=LinearRegression()
model_linear.fit(X_train, y_train)
y_pred_linear=model_linear.predict(X_test)
r2_score(y_test, y_pred_linear)

-2440.1064770476414

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.